In [1]:
import numpy
import btk
import os
import math
import re
from random import random

LoadError: numpy not found
while loading In[1], in expression starting on line 1

In [2]:
parent   = "/home/somebody/projects/20141104-rbohand2/"
filename = parent + "grasp6_1-RBOHand2_2.trb"

In [3]:
def get_positions(filename):
    reader = btk.btkAcquisitionFileReader() # build a btk reader object
    reader.SetFilename(filename) # set a filename to the reader
    reader.Update()
    acq = reader.GetOutput()
    
    frame_max =  acq.GetPointFrameNumber() #get the number of frames

    all_points = []
    for j in range(0, frame_max):
      points = {}
      for i in range(acq.GetPointNumber()):
        point = acq.GetPoint(i)
        label = point.GetLabel()
        values = point.GetValues() #returns a numpy array of the data, row=frame, col=coordinates
        points[label] = values[j][:]
        all_points.append(points)

    return all_points

In [4]:
data = get_positions(filename)

NameError: global name 'btk' is not defined

In [5]:
def get_domain_for_each_marker(data):
  domains = {}
  for key in data[0].keys():
    min_values = [data[0][key][i] for i in range(0,3)]
    max_values = [data[0][key][i] for i in range(0,3)]
    for j in range(1, len(data)):
      values = data[j][key]
      for i in range(0,3):
        if values[i] < min_values[i]:
          min_values[i] = values[i]
        if values[i] > max_values[i]:
          max_values[i] = values[i]
    domains[key] = [min_values, max_values]
  return domains

In [6]:
domains = get_domain_for_each_marker(data)

NameError: name 'data' is not defined

In [7]:
def walk(_dir, pattern, method):
    _dir = os.path.abspath(_dir)
    for _file in [_file for _file in os.listdir(_dir) if not _file in [".", "..", ".svn", ".git"]]:
        nfile = os.path.join(_dir, _file)
        if pattern.search(nfile):
            method(nfile)
        if os.path.isdir(nfile):
            walk(nfile, pattern, method)
                 

def get_domains_for_all_files(directory):
    pattern = re.compile(r".*RBOHand.*.trb$")
    files = []
    walk(directory, pattern, files.append)
    domains = [get_domain_for_each_marker(get_positions(filename)) for filename in files]
    global_domains = {}
    keys = domains[0].keys()
    for key in keys:
        domain = []
        for d in domains:
            minimum = d[key][0]
            maximum = d[key][1]
            if domain == []:
                domain = [ [minimum[0], minimum[1], minimum[2]], [maximum[0], maximum[1], maximum[2]]]

            d_min = [ domain[0][i] if minimum[i] > domain[0][i] else minimum[i] for i in range(0,3)]
            d_max = [ domain[1][i] if maximum[i] < domain[1][i] else maximum[i] for i in range(0,3)]
        global_domains[key] = [d_min, d_max]
    return global_domains

In [8]:
domains = get_domains_for_all_files(parent)

OSError: [Errno 2] No such file or directory: '/home/somebody/projects/20141104-rbohand2'

In [9]:
def scale_data_for_each_marker(data, domains):
  scaled_data = {}
  for key in domains.keys():
    new_data = []
    min_values = domains[key][0]
    max_values = domains[key][1]
    for j in range(0, len(data)):
      values = data[j][key]
      # if values[0] != nan:
      if max([abs((max_values[i] - min_values[i])) for i in range(0,3)]) > 0.00001:
        values = [(values[i] - min_values[i]) / (max_values[i] - min_values[i])
              for i in range(0,3)]
      new_data.append(values)
    scaled_data[key] = new_data
  return scaled_data

In [10]:
scaled_data = scale_data_for_each_marker(data, domains)

NameError: name 'data' is not defined

In [11]:
def bin_value(v, bins):
  return min(int(v * bins), bins-1)

def bin_vector(v, bins):
  return [min(int(v[i] * bins), bins-1) for i in range(0,3)]

def bin_scaled_data_for_each_marker(data, bins):
  new_data = {}
  for key in data.keys():
    new_data[key] = [bin_vector(v, bins) for v in data[key]]
  return new_data

In [12]:
binned_data = bin_scaled_data_for_each_marker(scaled_data, 10)

NameError: name 'scaled_data' is not defined

In [13]:
def combine_bin_vector(v, bins):
    return sum([v[i] * pow(bins,i) for i in range(0, len(v))])

def unique_valued_list(lst):
    myset = set(lst)
    return list(myset)

def relabel_vector(lst):
    mylst = unique_valued_list(lst)
    return [mylst.index(v) for v in lst]

def combine_bins_for_each_marker(data, bins):
    new_data = {}
    for key in data.keys():
        new_data[key] = relabel_vector([combine_bin_vector(v, bins) for v in data[key]])
    return new_data

def combine_random_variables(lst_of_lsts, bins):
    return relabel_vector([combine_bin_vector([v[i] for v in lst_of_lsts], bins) for i in range(0, len(lst_of_lsts[1]))])

In [14]:
# test combine_random_variables
list_1 = [0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9,1]
list_2 = [0,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,0,1]
list_3 = [0,1,2,3,4,5,6,7,8,9,2,3,4,5,6,7,8,9,0,1,1]

lst = [list_1, list_2, list_3]

combine_random_variables(lst, 10)

[0, 10, 18, 8, 16, 6, 14, 5, 13, 3, 12, 1, 11, 19, 9, 17, 7, 15, 2, 4, 10]

In [15]:
# creating unary data for 3D motion capturing data
combined_binned_data = combine_bins_for_each_marker(binned_data, 10)

NameError: name 'binned_data' is not defined

In [16]:
print max(combined_binned_data["2 PIP"]),
print max(combined_binned_data["2 DIP"]),
print max(combined_binned_data["3 MCP"])

46 37 2


In [17]:
def emperical_joint_distribution(w_prime, w, a):
  p = numpy.zeros((max(w_prime)+1, max(w)+1, max(a)+1))

  L = len(w_prime)
  for index in range(0, L):
    p[w_prime[index], w[index], a[index]] = p[w_prime[index], w[index], a[index]] + 1.0

  for i in range(0, p.shape[0]):
    for j in range(0, p.shape[1]):
      for k in range(0, p.shape[2]):
        p[i,j,k] = p[i,j,k] / float(L)

  s = sum(sum(sum(p)))
  p = p / s
  return p

In [18]:
jd = emperical_joint_distribution(combined_binned_data["2 PIP"],
                                  combined_binned_data["2 DIP"],
                                  combined_binned_data["3 MCP"])

In [19]:
def calc_p_w_prime_given_w(joint_distribution):
    p_w_prime_w = joint_distribution.sum(axis=2)
    p_w         = joint_distribution.sum(axis=(0,2))
    for w_prime in range(0,joint_distribution.shape[0]):
        for w in range(0, joint_distribution.shape[1]):
            p_w_prime_w[w_prime, w] = p_w_prime_w[w_prime, w] / p_w[w]
    return p_w_prime_w

def calc_p_w_prime_given_a(joint_distribution):
    p_w_prime_a = joint_distribution.sum(axis=1)
    p_a         = joint_distribution.sum(axis=(0,1))
    for w_prime in range(0,joint_distribution.shape[0]):
        for a in range(0, joint_distribution.shape[2]):
            if p_w_prime_a[w_prime, a] != 0.0 and p_a[a] != 0.0:
                p_w_prime_a[w_prime, a] = p_w_prime_a[w_prime, a] / p_a[a]
    return p_w_prime_a

def calc_p_w_prime_given_w_a(joint_distribution):
    p_w_a               = joint_distribution.sum(axis=0)
    p_w_prime_given_w_a = numpy.zeros(joint_distribution.shape)
    for w_prime in range(0, joint_distribution.shape[0]):
        for w in range(0, joint_distribution.shape[1]):
            for a in range(0, joint_distribution.shape[2]):
                if joint_distribution[w_prime, w, a] != 0.0 and p_w_a[w,a] != 0.0:
                    p_w_prime_given_w_a[w_prime, w, a] = joint_distribution[w_prime, w, a] / p_w_a[w,a]
    return p_w_prime_given_w_a

def calculate_concept_one(joint_distribution):
    p_w_prime_given_w   = calc_p_w_prime_given_w(joint_distribution)
    p_w_prime_given_w_a = calc_p_w_prime_given_w_a(joint_distribution)
    r = 0
    for w_prime in range(0, joint_distribution.shape[0]):
        for w in range(0, joint_distribution.shape[1]):
            for a in range(0, joint_distribution.shape[2]):
                if joint_distribution[w_prime, w, a] != 0.0 and p_w_prime_given_w[w_prime, w] != 0.0 and p_w_prime_given_w_a[w_prime, w, a] != 0.0:
                    r = joint_distribution[w_prime, w, a] * (math.log(p_w_prime_given_w_a[w_prime, w, a], 2) - math.log(p_w_prime_given_w[w_prime, w], 2))
    return r

def calculate_concept_two(joint_distribution):
    p_w_prime_given_a   = calc_p_w_prime_given_a(joint_distribution)
    p_w_prime_given_w_a = calc_p_w_prime_given_w_a(joint_distribution)
    r = 0
    for w_prime in range(0, joint_distribution.shape[0]):
        for w in range(0, joint_distribution.shape[1]):
            for a in range(0, joint_distribution.shape[2]):
                if joint_distribution[w_prime, w, a] != 0.0 and p_w_prime_given_a[w_prime, a] != 0.0 and p_w_prime_given_w_a[w_prime, w, a] != 0.0:
                    r = joint_distribution[w_prime, w, a] * (math.log(p_w_prime_given_w_a[w_prime, w, a], 2) - math.log(p_w_prime_given_a[w_prime, a], 2))
    return r

In [20]:
print calculate_concept_one(jd)
print calculate_concept_two(jd)

0.0
0.0377068913866


In [1]:
def calculate_bases(P):
    
    pass

## Work on data

In [28]:
def analyse_directory(parent, nr_of_bins, functions):
    print "reading all files and looking for their domains"
    domains     = get_domains_for_all_files(parent)
    
    binned_actions = None
    
    pattern = re.compile(r".*RBOHand.*.trb$")
    files = []
    walk(directory, pattern, files.append)
    
    print "Only using the first three files for test reasons."
    files = files[0:3]
    
    results = {}
    
    for f in files:
        print "reading file " + f
        data = get_positions(f)
        print "scaling data"
        scaled_data = scale_data_for_each_marker(data, domains)
        print "binning data"
        binned_data = bin_scaled_data_for_each_marker(scaled_data, nr_of_bins)
        print "combining data"
        combined_binned_data = combine_bins_for_each_marker(binned_data, nr_of_bins)
        combined_binned_data = combine_random_variables([combined_binned_data[key] for key in combined_binned_data.keys()], nr_of_bins)
        if binned_actions == None:
            print "randomising action data"
            binned_actions = [int(random() * nr_of_bins) for v in range(1,len(combined_binned_data))]
        print "calculate joint distribution"
        jd = emperical_joint_distribution(combined_binned_data[2:len(combined_binned_data)], combined_binned_data[1:len(combined_binned_data)-1], binned_actions)
        
        r = {}
        for key in functions.keys():
            print "using method: " + key
            r[key] = functions[key](jd)
        results[f] = r
    print "done."
    return results
    
    
directory = "/home/somebody/projects/20141104-rbohand2/"
bins      = 100
functions = {"One" : calculate_concept_one, "Two" : calculate_concept_two}

r = analyse_directory(directory, bins, functions)

reading all files and looking for their domains
Only using the first three files for test reasons.
reading file /home/somebody/projects/20141104-rbohand2/grasp1_11-RBOHand2_2.trb
scaling data
binning data
combining data
randomising action data
calculate joint distribution
using method: Two
using method: One
reading file /home/somebody/projects/20141104-rbohand2/grasp6_1-RBOHand2_2.trb
scaling data
binning data
combining data
calculate joint distribution
using method: Two
using method: One
reading file /home/somebody/projects/20141104-rbohand2/grasp4_1-RBOHand2_2.trb
scaling data
binning data
combining data
calculate joint distribution
using method: Two
using method: One
done.


In [29]:
print r

{'/home/somebody/projects/20141104-rbohand2/grasp6_1-RBOHand2_2.trb': {'Two': 0.0011403385336578151, 'One': 1.9815981196678177e-05}, '/home/somebody/projects/20141104-rbohand2/grasp1_11-RBOHand2_2.trb': {'Two': 0.00082464399025326113, 'One': 1.2739750870461339e-05}, '/home/somebody/projects/20141104-rbohand2/grasp4_1-RBOHand2_2.trb': {'Two': 0.00093404006948320361, 'One': 1.6389868854094223e-05}}


In [31]:
for key in r.keys():
    data = r[key]
    print key
    for k in r[key].keys():
        print "method: ",
        print k,
        print ": ",
        print r[key][k]

/home/somebody/projects/20141104-rbohand2/grasp6_1-RBOHand2_2.trb
method:  Two :  0.00114033853366
method:  One :  1.98159811967e-05
/home/somebody/projects/20141104-rbohand2/grasp1_11-RBOHand2_2.trb
method:  Two :  0.000824643990253
method:  One :  1.27397508705e-05
/home/somebody/projects/20141104-rbohand2/grasp4_1-RBOHand2_2.trb
method:  Two :  0.000934040069483
method:  One :  1.63898688541e-05
